In [11]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

In [17]:


# Définir le device (GPU ou CPU)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Chemins des dossiers d'images
image_root = './archive/Train'  # Dossier d'entraînement
test_root = './archive/Test'    # Dossier de test

# Charger les labels d'entraînement
train_csv = './archive/image_labels.csv'
train_labels = pd.read_csv(train_csv)

# Transformation des images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionner pour ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisation pour ResNet
])

In [19]:

# Dataset personnalisé
class ImageDataset(Dataset):
    def __init__(self, root, labels, transform=None):
        self.root = root
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, self.labels.iloc[idx]['Path'])
        image = Image.open(img_path).convert('RGB')  # Convertir en RGB
        label = self.labels.iloc[idx]['ClassId']

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:

# Créer les datasets et DataLoader
train_dataset = ImageDataset(image_root, train_labels, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

# Charger ResNet50 pré-entraîné
model = models.resnet50(pretrained=True)

# Remplacer la dernière couche fully connected pour s'adapter au nombre de classes
num_classes = 6  # Remplacez par le nombre de classes dans votre dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Déplacer le modèle sur le device (GPU ou CPU)
model = model.to(DEVICE)

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

C:\Users\pc\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\pc\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
# Fonction pour calculer la précision
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    correct = (preds == labels).sum().item()
    accuracy = 100.0 * correct / labels.size(0)
    return accuracy

In [ ]:






# Fonction d'entraînement
def train_model(model, train_loader, criterion, optimizer, num_epochs=10, device=DEVICE):
    model.train()
    train_loss_list = []
    train_acc_list = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Remettre les gradients à zéro
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass et optimisation
            loss.backward()
            optimizer.step()

            # Calculer les statistiques
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = 100.0 * running_corrects / len(train_loader.dataset)

        train_loss_list.append(epoch_loss)
        train_acc_list.append(epoch_acc.item())

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

    return train_loss_list, train_acc_list

# Entraîner le modèle
num_epochs = 10
train_loss_list, train_acc_list = train_model(model, train_loader, criterion, optimizer, num_epochs, DEVICE)

# Afficher les courbes de perte et de précision
plt.figure(figsize=(12, 5))

# Courbe de perte
plt.subplot(1, 2, 1)
plt.plot(train_loss_list, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

# Courbe de précision
plt.subplot(1, 2, 2)
plt.plot(train_acc_list, label='Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Training Accuracy')
plt.legend()

plt.show()

# Sauvegarder le modèle entraîné
torch.save(model.state_dict(), 'resnet50_model.pth')
print("Modèle sauvegardé avec succès.")

Epoch [1/10], Loss: 1.5317, Accuracy: 39.04%
